https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=VJ76KiP_dLn-

https://colab.research.google.com/drive/1uSlWtJdZmLrI3FCNIlUHFxwAJiSu2J0-

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math
from torch.jit import script, trace
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import tensorflow as tf
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [3]:
corpus_name = 'chatData'
corpus = os.path.join(r'C:\Users\admin\jupyter\pytorch\chatbot', corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding='utf-8') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, 'ko_chatdata.txt'))

$

잘 마실게.

오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..

무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.

하..

$

언니 과외 마쳤어요? 오늘은 늦게까지 했네요.

기말고사라..내가 하나 들어줘요?

괜찮아요.

$



In [4]:
def loadLines(fileName):
    lines = []
    count = 0
    count_list = []
    result = []
    with open(fileName, 'r', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            line = line.strip()
            if line == '$':
                count_list.append(count)
                count = 0
                pass
            else:
                lines.append(line)
                count+= 1
        count_list.append(count)
        count = 0
        for i in count_list: # 6, 4, 2, 7
            for j in range(count+1, count+i):
                result.append(lines[j-1] + ' [SEP] ' + lines[j])
            count += i
    return result
            
lines = loadLines(os.path.join(corpus, 'ko_chatdata.txt'))
lines[:5]

['잘 마실게. [SEP] 오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..',
 '오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서.. [SEP] 무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.',
 '무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래. [SEP] 하..',
 '언니 과외 마쳤어요? 오늘은 늦게까지 했네요. [SEP] 기말고사라..내가 하나 들어줘요?',
 '기말고사라..내가 하나 들어줘요? [SEP] 괜찮아요.']

In [5]:
datafile = os.path.join(corpus, 'bert_formatted_ko_conversations.txt')

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

# 띄어쓰기가 2번 입력됨
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n', quoting=csv.QUOTE_NONE, escapechar=' ')
    #writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in lines:
        writer.writerow([pair])

print("\nSample lines from file:")
printLines(datafile, n=3)
        
# 띄어쓰기 2번을 1번으로 바꿔서 다시 저장
print("\nRewriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    for pair in lines:
        outputfile.write(pair.replace('  ', ' '))
        outputfile.write('\n')
        
print("\nSample lines from file:")
printLines(datafile, n=3)


Writing newly formatted file...

Sample lines from file:
잘  마실게.  [SEP]  오후에  학부모  대표  모임에  사과하러  가는  거  교감선생님만  가시면  안돼요?  전  약속이  있어서..

오후에  학부모  대표  모임에  사과하러  가는  거  교감선생님만  가시면  안돼요?  전  약속이  있어서..  [SEP]  무슨  소리야?  누군  약속  없어서  거기  가?  사람  증말..무조건  같이  가.  사과  하루이틀해?  진짜  어이없어.이거  안  마실래.

무슨  소리야?  누군  약속  없어서  거기  가?  사람  증말..무조건  같이  가.  사과  하루이틀해?  진짜  어이없어.이거  안  마실래.  [SEP]  하..


Rewriting newly formatted file...

Sample lines from file:
잘 마실게. [SEP] 오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..

오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서.. [SEP] 무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.

무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래. [SEP] 하..



#from_pretrained + encode: 자동으로 CLS, SEP 삽입
sent = lines[0]
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.encode(sent)]
#tokenized_texts = [tokenizer.tokenize(line) for line in lines] # 그냥 기존 문장에 토큰화

#print(tokenizer.vocab)
#print(lines[0])
print(sent)
print(tokenized_texts[0])
for token in tokenized_texts:
    print(tokenizer.convert_ids_to_tokens(token))

tokens = tokenizer.convert_ids_to_tokens(tokenized_texts[0]) # convert_ids_to_tokens : 문장 토큰화

print(tokens)

tokens = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)

for token, id in zip(tokens, tokenized_texts[0]):
    if id == tokenizer.sep_token_id:
        print('')
    
    print('{:<12} {:>6,}'.format(token, id)) 
    
    if id == tokenizer.sep_token_id:
        print('')

# `[SEP]`토큰의 첫 번째 인스턴스에 대한 input_ids를 검색합니다

sep_index = []

for sentence in tokenized_texts:
    sep_index.append([sentence.index(tokenizer.sep_token_id)]) # sep_token_id : 토큰화된 id가 102번(SEP)일때 
    continue

print(sep_index[0])
print(len(sep_index))
print(len(tokenized_texts))

In [6]:
PAD_token = 0
SOS_token = 1
EOS_token = 2
CLS_token = 101
SEP_token = 102
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token : 'PAD', SOS_token:'SOS', EOS_token:'EOS', CLS_token : 'CLS', SEP_token : 'SEP'}
        self.num_words = 5
        self.tokenizer = tokenizer
        self.tokens = []
        
    def addSentence(self, sentence): # sentence : 잘 마실게
        sentence = self.tokenizer.encode(sentence)
        tokens = self.tokenizer.convert_ids_to_tokens(sentence)
        self.tokens = tokens
        # 잘 마실게. [SEP] 오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..
        #for word in tokens:
            #self.addWord(word)
        #print(tokens)
        return tokens            

In [7]:
MAX_LENGTH = 10  

def readVocs(datafile, corpus_name): # corpus_name : chatData / datafile : formatted_ko_conversations.txt
    print('Reading lines...')
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [l.strip().split(' [SEP] ') for l in lines]
    #print(pairs[0])
    voc = Voc(corpus_name)
    return voc, pairs # voc : 문서 단어집합 / pairs : 문장 쌍 집합

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name) # voc : 단어집합, pairs : 질문 쌍
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    for pair in pairs:
        sentence1 = voc.addSentence(pair[0].strip())
        #print(sentence1)
        sentence2 = voc.addSentence(pair[1].strip())
    return voc, pairs

save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 70560 sentence pairs
Trimmed to 42241 sentence pairs
['언니 과외 마쳤어요? 오늘은 늦게까지 했네요.', '기말고사라..내가 하나 들어줘요?']
['기말고사라..내가 하나 들어줘요?', '괜찮아요.']
['스트레스 받는 애 성적이 저 모양이야?', '지도 잘하고 싶겠지. 지라고 못하고 싶겠어?']
['내가 보기엔', '나가서 냉수 한잔만 떠다줘. 아우. 괜히 또 열받네.']
['나가서 냉수 한잔만 떠다줘. 아우. 괜히 또 열받네.', '내 생각엔']
['내 생각엔', '빨리 냉수 좀 떠줘.']
['빨리 냉수 좀 떠줘.', '알았어.']
['이순팔 맞는데..', '아깐 자기가 이순댄가 뭐그래놓구는..']
['아깐 자기가 이순댄가 뭐그래놓구는..', '이분 진짜 이름이 뭐예요?']
['이분 진짜 이름이 뭐예요?', '본인이 이순팔이라면 이순팔이겠죠 뭐. 전 가도 돼죠?']


In [8]:
def indexesFromSentence(voc, sentence):
    #tokens = tokenizer.tokenize(sentence)
    return tokenizer.encode(sentence)

def zeroPadding(l, fillvalue=PAD_token):
    # print(list(itertools.zip_longest(*l, fillvalue=fillvalue))) : (1957, 2, 2, 2, 0)
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc): # l : 응. 새벽에도 일찍 나갔어. 온다간다 말도없이.
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l] 
    # print(indexes_batch) : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2], [319, 2], [186, 2]]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch]) 
    # print(lengths) : tensor([4, 4, 3, 2, 2])
    padList = zeroPadding(indexes_batch)
    # print(padList) : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2, 0], [319, 2, 0, 0], [186, 2, 0, 0]]
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    #print(indexes_batch)
    max_target_len = max([len(indexes) for indexes in indexes_batch]) 
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList) 
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(tokenizer.tokenize(x[0])), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[   101,    101,    101,    101,    101],
        [  9553,   9389,   9524,   9303,   9576],
        [ 23811,  62200, 119118,  11287,    119],
        [  9596, 119009,  12965,    136,    102],
        [   136,  62200,    119,    102,      0],
        [  9706,  27023,    102,      0,      0],
        [ 40032,   9960,      0,      0,      0],
        [ 38401,    119,      0,      0,      0],
        [  9580,    102,      0,      0,      0],
        [ 59894,      0,      0,      0,      0],
        [   136,      0,      0,      0,      0],
        [   102,      0,      0,      0,      0]])
lengths: tensor([12,  9,  6,  5,  4])
target_variable: tensor([[   101,    101,    101,    101,    101],
        [  9641,   9519,   9604,   9004,   9672],
        [ 82888,   9574,  12692,   9596,  11287],
        [ 10739, 105197,   9987,   8982,   9546],
        [ 17342,   9653,  15891,  11102,  17730],
        [ 12092,  22333,   9638, 119353,   9954],
        [  9638,  12692, 119

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, 
                          dropout = (0 if n_layers == 1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq) # input_seq : shape=(max_length, batch_size), input_variable
        # print(input_lengths) : =lengths
        # print(embedded.shape) : torch.Size([10, 64, 500]) [max_length, batch_size, hidden_size(은닉상태 크기)]
        
        # nn.utils.rnn.pack_padded_sequence : 패딩연산처리 쉽게하기 위해 중간에 빈공간 제거(형태 : tensor)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths) # input_lengths : shape=(batch_size)
        #print(packed.batch_sizes)
        # print(packed.batch_sizes) : tensor([64, 64, 64, 58, 52, 45, 38, 17,  8,  2])
        
        outputs, hidden = self.gru(packed, hidden) # 입력hidden : shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(outputs.batch_sizes) : tensor([64, 64, 63, 52, 47, 34, 24, 18, 12,  6])
        # print(hidden.shape) : torch.Size([4, 64, 500]) [층 * 양방향이면2 아니면1, batch_size, hidden_size]
        
        # nn.utils.rnn.pad_packed_sequence : 패딩연산이 끝난 것을 다시 원래대로 (형태 : torch)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        #print(outputs.shape)
        # print(outputs.shape)# : torch.Size([10, 64, 1000]) # [max_length, batch_size, hidden_size(양방향으로 진행했으면 *2)]
        
        # 양방향 GRU의 출력을 합산합니다
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # print(outputs.shape) : torch.Size([10, 64, 500])
        
        # hidden : GRU의 최종 은닉 상태
        return outputs, hidden

In [10]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, 'is not an appropriate attention method.')
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            
    # 가중치 계산을 dot-product로 계산
    def dot_score(self, hidden, encoder_output):
        # print(torch.sum(hidden * encoder_output, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * encoder_output, dim=2)
    
    # 
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        
        # print(torch.sum(hidden * energy, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * energy, dim=2)
    
    
    def concat_score(self, hidden, encoder_output):
        # cat : 합칠 때 차원은 2차원으로 / expand : 확장
        # Tanh 함수는 함수값을 [-1, 1]로 제한시킴
        # print((hidden.expand(encoder_output.size(0), -1, -1).shape)) : torch.Size([10, 64, 500])
        # print(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2).shape) : torch.Size([10, 64, 1000])
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
            
        attn_energies = attn_energies.t() # t() : 행과 열을 바꿔서 저장[1, 2, 3], [4, 5, 6] -> [1, 4, 7], [2, 5, 8]
        
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [11]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 참조를 보존해 둡니다
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 레이어를 정의합니다
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행합니다
        # 현재의 입력 단어에 대한 임베딩을 구합니다   
        #print(input_step)
        embedded = self.embedding(input_step) # input_step : 입력 시퀀스 배치에 대한 한 단위 시간(한 단어). shape=(1, batch_size)
        embedded = self.embedding_dropout(embedded)
        #print(embedded.shape)
        # print(embedded.shape) : torch.Size([1, 64, 500])
        
        # 양방향x
        # last_hidden : GRU의 마지막 은닉 레이어. shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(last_hidden.shape) : torch.Size([2, 64, 500]) 
        rnn_output, hidden = self.gru(embedded, last_hidden) 
        # print(rnn_output.shape) : torch.Size([1, 64, 500])
        # print(hidden.shape) : torch.Size([2, 64, 500])

        # attention 가중치
        attn_weights = self.attn(rnn_output, encoder_outputs) # encoder_outputs : 인코더 모델 출력 shape=(max_length, batch_size, hidden_size)
        # print(attn_weights.shape) : torch.Size([64, 1, 10]) 

        # 인코더 출력에 어텐션을 곱하여 새로운 context vector생성
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # print(context.shape) : torch.Size([64, 1, 500])

        rnn_output = rnn_output.squeeze(0) # print(rnn_output.shape) : torch.Size([64, 500])
        context = context.squeeze(1) # print(context.shape) : torch.Size([64, 500])
        concat_input = torch.cat((rnn_output, context), 1) # print(concat_input.shape) : torch.Size([64, 1000])
        concat_output = torch.tanh(self.concat(concat_input))
        # print(concat_output.shape) : torch.Size([64, 500])

        # output : 각 단어가 디코딩된 시퀀스에서 다음 단어로 사용되었을 때 적합할 확률을 나타내는 정규화된 softmax 텐서. 
        # shape=(batch_size, voc.num_words)
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden

model_name = 'cb_model'
attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

embedding = nn.Embedding(len(tokenizer.vocab), hidden_size)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(tokenizer.vocab), decoder_n_layers, dropout)

encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

decoder_hidden = encoder_hidden[:decoder.n_layers]
#print(decoder_hidden.shape)
decoder_input = torch.ones(1, 1, dtype=torch.long) * CLS_token
#print(decoder_input.shape)

#embedded = embedding(decoder_input) # input_step : 입력 시퀀스 배치에 대한 한 단위 시간(한 단어). shape=(1, batch_size)
# embedded = embedding_dropout(decoder_input)
# print(embedded.shape)

# 이부분 오류>>
decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)

In [12]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [13]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer,
         batch_size, clip, max_length = MAX_LENGTH):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    
    loss = 0
    print_losses = []
    n_totals = 0
    
    # EncoderRNN의 forward부분 실행
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # 초기 디코더 입력을 생성(각 문장을 SOS 토큰으로 시작)
    decoder_input = torch.LongTensor([[CLS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    # 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    # teacher_forcing : Decoder부분에서 앞 단어가 잘못 추측되었을 경우 뒤에도 달라지니 정답을 입력해 주는 것
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for t in range(max_target_len):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
             # Teacher forcing 사용: 다음 입력을 현재의 목표로 둡니다
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 둡니다
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
            
    loss.backward()
    
    # clip_grad_norm_: 그라디언트를 제자리에서 수정합니다
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses) / n_totals

In [14]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 각 단계에 대한 배치 설정
    # batch2TrainData : return inp, lengths, output, mask, max_target_len
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]


    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1


    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        torch.cuda.empty_cache() # GPU 캐시 데이터 삭제
        training_batch = training_batches[iteration - 1]
        
        input_variable, lengths, target_variable, mask, max_target_len = training_batch


        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        
        print_loss += loss


        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Checkpoint를 저장
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [15]:
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 32


loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# loadFilename이 제공되는 경우에는 모델을 불러옵니다
if loadFilename:
    # 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    # GPU에서 학습한 모델을 CPU로 불러오는 경우
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')

embedding = nn.Embedding(len(tokenizer.vocab), hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(tokenizer.vocab), decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [16]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 1200
print_every = 1
save_every = 500

# Dropout 레이어를 학습 모드로 둡니다
encoder.train()
decoder.train()

# Optimizer를 초기화합니다
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# 학습 단계를 수행합니다
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


<ipython-input-12-f319a975feb5>:4: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:20.)
  loss = crossEntropy.masked_select(mask).mean()


Iteration: 1; Percent complete: 0.1%; Average loss: 11.7010
Iteration: 2; Percent complete: 0.1%; Average loss: 11.6465
Iteration: 3; Percent complete: 0.2%; Average loss: 11.5750
Iteration: 4; Percent complete: 0.3%; Average loss: 11.4771
Iteration: 5; Percent complete: 0.3%; Average loss: 11.2973
Iteration: 6; Percent complete: 0.4%; Average loss: 11.0245
Iteration: 7; Percent complete: 0.5%; Average loss: 10.6935
Iteration: 8; Percent complete: 0.5%; Average loss: 10.0637
Iteration: 9; Percent complete: 0.6%; Average loss: 9.5047
Iteration: 10; Percent complete: 0.7%; Average loss: 9.0768
Iteration: 11; Percent complete: 0.7%; Average loss: 8.6005
Iteration: 12; Percent complete: 0.8%; Average loss: 8.7577
Iteration: 13; Percent complete: 0.9%; Average loss: 8.3497
Iteration: 14; Percent complete: 0.9%; Average loss: 7.8889
Iteration: 15; Percent complete: 1.0%; Average loss: 7.5788
Iteration: 16; Percent complete: 1.1%; Average loss: 7.3735
Iteration: 17; Percent complete: 1.1%; Av

Iteration: 137; Percent complete: 9.1%; Average loss: 5.1258
Iteration: 138; Percent complete: 9.2%; Average loss: 5.0816
Iteration: 139; Percent complete: 9.3%; Average loss: 5.4520
Iteration: 140; Percent complete: 9.3%; Average loss: 4.8946
Iteration: 141; Percent complete: 9.4%; Average loss: 5.1522
Iteration: 142; Percent complete: 9.5%; Average loss: 4.9715
Iteration: 143; Percent complete: 9.5%; Average loss: 5.1565
Iteration: 144; Percent complete: 9.6%; Average loss: 5.3152
Iteration: 145; Percent complete: 9.7%; Average loss: 5.0822
Iteration: 146; Percent complete: 9.7%; Average loss: 4.9335
Iteration: 147; Percent complete: 9.8%; Average loss: 4.9546
Iteration: 148; Percent complete: 9.9%; Average loss: 4.9358
Iteration: 149; Percent complete: 9.9%; Average loss: 4.9845
Iteration: 150; Percent complete: 10.0%; Average loss: 5.2767
Iteration: 151; Percent complete: 10.1%; Average loss: 5.0613
Iteration: 152; Percent complete: 10.1%; Average loss: 5.0710
Iteration: 153; Perce

Iteration: 270; Percent complete: 18.0%; Average loss: 4.5852
Iteration: 271; Percent complete: 18.1%; Average loss: 4.8524
Iteration: 272; Percent complete: 18.1%; Average loss: 4.7695
Iteration: 273; Percent complete: 18.2%; Average loss: 5.2061
Iteration: 274; Percent complete: 18.3%; Average loss: 4.3556
Iteration: 275; Percent complete: 18.3%; Average loss: 4.5217
Iteration: 276; Percent complete: 18.4%; Average loss: 4.6444
Iteration: 277; Percent complete: 18.5%; Average loss: 4.5194
Iteration: 278; Percent complete: 18.5%; Average loss: 4.8012
Iteration: 279; Percent complete: 18.6%; Average loss: 4.9267
Iteration: 280; Percent complete: 18.7%; Average loss: 4.6209
Iteration: 281; Percent complete: 18.7%; Average loss: 4.6996
Iteration: 282; Percent complete: 18.8%; Average loss: 5.0008
Iteration: 283; Percent complete: 18.9%; Average loss: 4.8027
Iteration: 284; Percent complete: 18.9%; Average loss: 4.9050
Iteration: 285; Percent complete: 19.0%; Average loss: 4.8601
Iteratio

...

Iteration: 935; Percent complete: 62.3%; Average loss: 3.9193
Iteration: 936; Percent complete: 62.4%; Average loss: 3.7644
Iteration: 937; Percent complete: 62.5%; Average loss: 4.1565
Iteration: 938; Percent complete: 62.5%; Average loss: 3.6134
Iteration: 939; Percent complete: 62.6%; Average loss: 3.5167
Iteration: 940; Percent complete: 62.7%; Average loss: 3.8804
Iteration: 941; Percent complete: 62.7%; Average loss: 3.7159
Iteration: 942; Percent complete: 62.8%; Average loss: 3.9817
Iteration: 943; Percent complete: 62.9%; Average loss: 4.0929
Iteration: 944; Percent complete: 62.9%; Average loss: 3.9385
Iteration: 945; Percent complete: 63.0%; Average loss: 3.6761
Iteration: 946; Percent complete: 63.1%; Average loss: 3.7380
Iteration: 947; Percent complete: 63.1%; Average loss: 3.8339
Iteration: 948; Percent complete: 63.2%; Average loss: 3.6718
Iteration: 949; Percent complete: 63.3%; Average loss: 3.8362
Iteration: 950; Percent complete: 63.3%; Average loss: 3.9787
Iteratio

Iteration: 1067; Percent complete: 71.1%; Average loss: 3.8293
Iteration: 1068; Percent complete: 71.2%; Average loss: 3.6585
Iteration: 1069; Percent complete: 71.3%; Average loss: 3.9370
Iteration: 1070; Percent complete: 71.3%; Average loss: 3.4136
Iteration: 1071; Percent complete: 71.4%; Average loss: 4.0223
Iteration: 1072; Percent complete: 71.5%; Average loss: 3.5873
Iteration: 1073; Percent complete: 71.5%; Average loss: 3.7074
Iteration: 1074; Percent complete: 71.6%; Average loss: 4.1669
Iteration: 1075; Percent complete: 71.7%; Average loss: 3.8386
Iteration: 1076; Percent complete: 71.7%; Average loss: 3.6175
Iteration: 1077; Percent complete: 71.8%; Average loss: 3.9454
Iteration: 1078; Percent complete: 71.9%; Average loss: 3.7987
Iteration: 1079; Percent complete: 71.9%; Average loss: 3.8188
Iteration: 1080; Percent complete: 72.0%; Average loss: 3.7995
Iteration: 1081; Percent complete: 72.1%; Average loss: 3.7135
Iteration: 1082; Percent complete: 72.1%; Average loss:

Iteration: 1198; Percent complete: 79.9%; Average loss: 3.8077
Iteration: 1199; Percent complete: 79.9%; Average loss: 3.5284
Iteration: 1200; Percent complete: 80.0%; Average loss: 3.5886
Iteration: 1201; Percent complete: 80.1%; Average loss: 3.4848
Iteration: 1202; Percent complete: 80.1%; Average loss: 3.7389
Iteration: 1203; Percent complete: 80.2%; Average loss: 3.3724
Iteration: 1204; Percent complete: 80.3%; Average loss: 3.7866
Iteration: 1205; Percent complete: 80.3%; Average loss: 3.3426
Iteration: 1206; Percent complete: 80.4%; Average loss: 3.3954
Iteration: 1207; Percent complete: 80.5%; Average loss: 3.4642
Iteration: 1208; Percent complete: 80.5%; Average loss: 3.8039
Iteration: 1209; Percent complete: 80.6%; Average loss: 3.5071
Iteration: 1210; Percent complete: 80.7%; Average loss: 3.6352
Iteration: 1211; Percent complete: 80.7%; Average loss: 3.5297
Iteration: 1212; Percent complete: 80.8%; Average loss: 3.8286
Iteration: 1213; Percent complete: 80.9%; Average loss:

Iteration: 1329; Percent complete: 88.6%; Average loss: 3.4511
Iteration: 1330; Percent complete: 88.7%; Average loss: 3.6300
Iteration: 1331; Percent complete: 88.7%; Average loss: 3.7559
Iteration: 1332; Percent complete: 88.8%; Average loss: 3.7384
Iteration: 1333; Percent complete: 88.9%; Average loss: 3.4431
Iteration: 1334; Percent complete: 88.9%; Average loss: 3.4550
Iteration: 1335; Percent complete: 89.0%; Average loss: 3.6776
Iteration: 1336; Percent complete: 89.1%; Average loss: 3.4613
Iteration: 1337; Percent complete: 89.1%; Average loss: 3.5645
Iteration: 1338; Percent complete: 89.2%; Average loss: 3.3287
Iteration: 1339; Percent complete: 89.3%; Average loss: 3.2490
Iteration: 1340; Percent complete: 89.3%; Average loss: 3.5965
Iteration: 1341; Percent complete: 89.4%; Average loss: 3.9582
Iteration: 1342; Percent complete: 89.5%; Average loss: 3.3142
Iteration: 1343; Percent complete: 89.5%; Average loss: 3.6647
Iteration: 1344; Percent complete: 89.6%; Average loss:

Iteration: 1460; Percent complete: 97.3%; Average loss: 3.4260
Iteration: 1461; Percent complete: 97.4%; Average loss: 3.3558
Iteration: 1462; Percent complete: 97.5%; Average loss: 3.3268
Iteration: 1463; Percent complete: 97.5%; Average loss: 3.5181
Iteration: 1464; Percent complete: 97.6%; Average loss: 3.4729
Iteration: 1465; Percent complete: 97.7%; Average loss: 3.6682
Iteration: 1466; Percent complete: 97.7%; Average loss: 3.6181
Iteration: 1467; Percent complete: 97.8%; Average loss: 3.6999
Iteration: 1468; Percent complete: 97.9%; Average loss: 3.4920
Iteration: 1469; Percent complete: 97.9%; Average loss: 3.3328
Iteration: 1470; Percent complete: 98.0%; Average loss: 3.7971
Iteration: 1471; Percent complete: 98.1%; Average loss: 3.5036
Iteration: 1472; Percent complete: 98.1%; Average loss: 3.7385
Iteration: 1473; Percent complete: 98.2%; Average loss: 3.4050
Iteration: 1474; Percent complete: 98.3%; Average loss: 3.6556
Iteration: 1475; Percent complete: 98.3%; Average loss:

In [17]:
# 탐욕적 디코딩(Greedy decoding) : 각 단계에 대해 단순히 decoder_output 에서 가장 높은 softmax값을 갖는 단어를 선택하는 방식
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):

        # EncoderRNN의 forward부분 실행
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        #print('outputs : ', encoder_outputs.shape)
        #print('hidden : ', encoder_hidden.shape)
               
        # encoder의 마지막 hidden이 decoder의 처음 hidden
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        #print('decoder_hidden : ', decoder_hidden.shape)
        
        # decoder의 처음입력을 SOS로 초기화
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * CLS_token
        #print('decoder_input : ', decoder_input)
        
        # 디코더가 단어를 덧붙여 나갈 텐서를 초기화
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)

        for _ in range(max_length):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            #print('decoder_output : ', decoder_output.shape)
            #print('decoder_hidden : ', decoder_hidden.shape)
            
            # 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구합니다
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            #print('decoder_scores : ', decoder_scores)
            #print('decoder_input : ', decoder_input)

            # 토큰, 점수 기록
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)

            # 현재의 토큰을 디코더의 다음 입력으로 준비시킵니다(차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)

        return all_tokens, all_scores

In [18]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # indexes_batch : 문장을 단어집합에 저장된 수로 바꾼후 마지막에 EOS추가하는 함수
    indexes_batch = [tokenizer.encode(sentence)]
    #print('input : ', indexes_batch)
    
    # lengths 텐서를 만듭니다
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    #print(lengths)
    
    # 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만듭니다
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    #print('input : ', input_batch)
    
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    
    # searcher를 이용하여 문장을 디코딩합니다
    G_tokens, scores = searcher(input_batch, lengths, max_length)
    #print(G_tokens)
    #print(scores)

    # 인덱스 -> 단어    
    #print([tokenizer.convert_ids_to_tokens(token) for token in G_tokens])
#     for token in G_tokens:
#         print(tokenizer.convert_ids_to_tokens([token]))
    decoded_words = tokenizer.convert_ids_to_tokens(G_tokens) ###수정해야함
    #print(decoded_words)
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 입력 문장을 받아옵니다
            input_sentence = input('> ')
            # 종료 조건인지 검사합니다
            if input_sentence == 'exit' or input_sentence == '종료': break
            # 문장을 평가합니다
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 응답 문장을 형식에 맞춰 출력합니다
            output_words[:] = [x for x in output_words if not (x == '[CLS]' or x == '[SEP]' or x == 'PAD' or x == 'SOS' or x == 'EOS')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [19]:
# Dropout 레이어를 평가 모드로 설정합니다
encoder.eval()
decoder.eval()

# 탐색 모듈을 초기화합니다
searcher = GreedySearchDecoder(encoder, decoder)

# 채팅을 시작합니다 (다음 줄의 주석을 제거하면 시작해볼 수 있습니다)
evaluateInput(encoder, decoder, searcher, voc)

> 안녕
Bot: 아 ##니 . . ##마 . ##마
> 왜그래ㅠ
Bot: 그 ##래 ##도 [UNK] . ##가 [UNK] .
> 왜 대답이
Bot: 그 ##래 . . . .
> 이렇게 나오는거야
Bot: 그 ##래 ? ##고 . ##마 .
> 어디야
Bot: 그 ##럼 . . . .
> 어디야?
Bot: 그 ##럼 . . . .
> 대답이 왜 
Bot: 그 ##래 . ##마 . ##마 .
> 왜 중복으로 나오는거야?
Bot: 아 ##니 . . . ##고 .
> 짜증나
Bot: 그 ##래 ##도 아 ##니 ##에 ##요 .
> 화나
Bot: 그 ##래 ##도 아 ##니 . ##마 .
> 무슨 말이라도 해봐
Bot: 그 ##래 ##도 그 ##래 ##도 그 ##래 ##도


KeyboardInterrupt: Interrupted by user